In [1]:
import os
import errno

import pandas as pd
import numpy as np

import deepcell

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from deepcell.utils.plot_utils import get_js_video
from IPython.display import HTML


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/usr/local/lib/python3.6/dist-packages/deepcell/utils/__init__.py:49: UserWarning: To use `compute_overlap`, the C extensions must be built using `python setup.py build_ext --inplace`
  warnings.warn('To use `compute_overlap`, the C extensions must be built '


In [2]:
# Load data - note, /data is mounted to /images
# normalize raw
from deepcell_toolbox.processing import phase_preprocess

filepath = '/images/data/cells/MouseBrain/3D_annotations/'
set_nums = 2

X = []
y = []

for set_num in range(set_nums):
    path_to_npz = os.path.join(filepath, 'mov_{}'.format(set_num), 'mov_{}_edited.npz'.format(set_num))

    with np.load(path_to_npz) as load_data:
        X_set = load_data['X']
        y_set = load_data['y']
        
    
    X.append(np.expand_dims(X_set[..., 0], axis=-1))
    y.append(y_set)
    
X = np.asarray(X)
y = np.asarray(y)


for frame in range(X.shape[1]):

    X[:, frame, ...] = phase_preprocess(X[:, frame, ...])

print('Shape of X is {} and shape of y is {}'.format(X.shape, y.shape))





Shape of X is (2, 47, 1024, 1024, 1) and shape of y is (2, 47, 1024, 1024, 1)


In [3]:
from deepcell_toolbox.utils import tile_image_3D, untile_image_3D

im_shape = (21, 128, 128)
stride_ratio = 0.75

X_ims, tiles_info_X = tile_image_3D(X, model_input_shape=im_shape, stride_ratio=stride_ratio)
y_ims, tiles_info_y = tile_image_3D(y, model_input_shape=im_shape, stride_ratio=stride_ratio)

print('Shape of X_ims is {} and shape of y_ims is {}'.format(X_ims.shape, y_ims.shape))

Shape of X_ims is (726, 21, 128, 128, 1) and shape of y_ims is (726, 21, 128, 128, 1)


In [4]:
"""Utilities for data transformations"""

from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import numpy as np
from scipy import ndimage
from skimage.measure import label
from skimage.measure import regionprops
from skimage.morphology import ball, disk
from skimage.morphology import binary_erosion, binary_dilation
from tensorflow.python.keras import backend as K

from deepcell_toolbox import erode_edges

def inner_distance_transform_3d(mask, bins=None,
                                erosion_width=None,
                                alpha=0.1, beta=1,
                                sampling=[0.5, 0.217, 0.217]):
    """Transform a label mask for a z-stack with an inner distance transform.
    inner_distance = 1 / (1 + beta * alpha * distance_to_center)

    Args:
        mask (numpy.array): A label mask (y data).
        bins (int): The number of transformed distance classes.
            Defaults to None.
        erosion_width (int): Number of pixels to erode edges of each labels
        alpha (float, str): Coefficent to reduce the magnitude of the distance
            value. If 'auto', determines alpha for each cell based on the cell
            area. Defaults to 0.1.
        beta (float): Scale parameter that is used when alpha is set to auto.
            Defaults to 1.
        sampling (list): Spacing of pixels along each dimension.
            Defaults to [0.5, 0.217, 0.217].

    Returns:
        numpy.array: A mask of same shape as input mask,
            with each label being a distance class from 1 to bins.

    Raises:
        ValueError: alpha is a string but not set to "auto".
    """

    # Check input to alpha
    if isinstance(alpha, str):
        if alpha.lower() != 'auto':
            raise ValueError('alpha must be set to "auto"')

    mask = np.squeeze(mask)
    mask = erode_edges(mask, erosion_width)

    # Pad along z axis
#    num_frames = mask.shape[0]
#    mask = np.pad(mask, ((1,1), (0,0), (0,0)))

    distance = ndimage.distance_transform_edt(mask, sampling=sampling)
    distance = distance.astype(K.floatx())

    label_matrix = label(mask)

    inner_distance = np.zeros(distance.shape, dtype=K.floatx())
    for prop in regionprops(label_matrix, distance):
        coords = prop.coords
        center = prop.weighted_centroid
        distance_to_center = (coords - center) * np.array(sampling)
        distance_to_center = np.sum(distance_to_center ** 2, axis=1)

        # Determine alpha to use
        if str(alpha).lower() == 'auto':
            _alpha = 1 / np.cbrt(prop.area)
        else:
            _alpha = float(alpha)

        center_transform = 1 / (1 + beta * _alpha * distance_to_center)
        coords_z = coords[:, 0]
        coords_x = coords[:, 1]
        coords_y = coords[:, 2]
        inner_distance[coords_z, coords_x, coords_y] = center_transform

    # Undo padding
#    inner_distance = inner_distance[1:num_frames+1, ...]

    if bins is None:
        return inner_distance
    
def outer_distance_transform_2d(mask, bins=None, erosion_width=None,
                                normalize=True):
    """Transform a label mask with an outer distance transform.

    Args:
        mask (numpy.array): A label mask (y data).
        bins (int): The number of transformed distance classes. If none,
            returns the continuous outer transform.
        erosion_width (int): Number of pixels to erode edges of each labels
        normalize (boolean): Normalize the transform of each cell by that
            cell's largest distance.

    Returns:
        numpy.array: A mask of same shape as input mask,
            with each label being a distance class from 1 to bins.
    """
    mask = np.squeeze(mask)  # squeeze the channels
    mask = erode_edges(mask, erosion_width)

    distance = ndimage.distance_transform_edt(mask)
    distance = distance.astype(K.floatx())  # normalized distances are floats

    if normalize:
        # uniquely label each cell and normalize the distance values
        # by that cells maximum distance value
        label_matrix = label(mask)
        for prop in regionprops(label_matrix):
            labeled_distance = distance[label_matrix == prop.label]
            normalized_distance = labeled_distance / np.amax(labeled_distance)
            distance[label_matrix == prop.label] = normalized_distance

    if bins is None:
        return distance
    
def outer_distance_transform_3d(mask, bins=None, erosion_width=None,
                                normalize=True, sampling=[0.5, 0.217, 0.217]):
    """Transforms a label mask for a z stack with an outer distance transform.
    Uses scipy's distance_transform_edt

    Args:
        mask (numpy.array): A z-stack of label masks (y data).
        bins (int): The number of transformed distance classes.
            Defaults to None.
        erosion_width (int): Number of pixels to erode edges of each labels.
            Defaults to None.
        normalize (boolean): Normalize the transform of each cell by that
            cell's largest distance. Defaults to True.
        sampling (list): Spacing of pixels along each dimension.
            Defaults to [0.5, 0.217, 0.217].

    Returns:
        numpy.array: 3D Euclidiean Distance Transform
    """
    maskstack = np.squeeze(mask)  # squeeze the channels
    maskstack = erode_edges(maskstack, erosion_width)

#    print('Maskstack shape in outer_dist is {}'.format(maskstack.shape))

    distance = ndimage.distance_transform_edt(maskstack, sampling=sampling)

    # normalize by maximum distance
    if normalize:
        for cell_label in np.unique(maskstack):
            if cell_label == 0:  # distance is only found for non-zero regions
                continue
            index = np.nonzero(maskstack == cell_label)
            distance[index] = distance[index] / np.amax(distance[index])

    if bins is None:
        return distance

In [5]:
test_tile = np.copy(y_ims[160, ...])
mask_stack = np.where(test_tile != 221, 0, test_tile)
mask_stack = np.expand_dims(mask_stack, 0)
np.unique(test_tile)

array([  0,  62,  65,  67, 221, 257, 311], dtype=uint16)

In [6]:
mask_stack = outer_distance_transform_3d(mask_stack)
mask_stack = np.expand_dims(np.expand_dims(mask_stack, axis=0), axis=-1)
mask_stack*=20 # scale for visibility

In [8]:
HTML(get_js_video(mask_stack, batch=0, channel=0, interval=500))

In [ ]:
HTML(get_js_video(y_ims, batch=160, channel=0, interval=500, cmap='jet', vmin=0, vmax=y_ims.max()))